In [1]:
import sys; sys.path.append('../util')
from load_yelp_data import load_yelp_dataframe, restaurants_and_bars_in, train_test_split_reviews
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics
import scipy.optimize
%matplotlib inline

from baseline_calculator import *

In [2]:
businesses = load_yelp_dataframe('businesses')
reviews = load_yelp_dataframe('reviews')
users = load_yelp_dataframe('users')

In [3]:
phoenix_restaurants, phoenix_reviews, phoenix_users = restaurants_and_bars_in('Phoenix', businesses, reviews, users)

In [4]:
reviews_train, reviews_test = train_test_split_reviews(phoenix_reviews)

In [5]:
bc = L2RegLeastSquaresBaselineCalculator()
bc.fit(reviews_train)

828243.574404
820873.878045
814394.684473
808624.424522
803428.02802
798703.626399
794373.295229
790376.562701
786665.822104
783203.062639
779957.518782
776903.964011
774021.459808
771292.428842
768701.960885
766237.287241
763887.378337
761642.632134
759494.630209
757435.944728
755459.984103
753560.868332
751733.327358
749972.617459
748274.451873
746634.942808
745050.552587
743518.052239
742034.486165
740597.141817
739203.523547
737851.329934
736538.43404
735262.866145
734022.798586
732816.532403
731642.485523
730499.182288
729385.244127
728299.381244
727240.385173
726207.122103
725198.526878
724213.597585
723251.390671
722311.016518
721391.63544
720492.45403
719612.721849
718751.728401
717908.800365
717083.299075
716274.618193
715482.181592
714705.441395
713943.876178
713196.989312
712464.307434
711745.379028
711039.773123
710347.078073
709666.90044
708998.863944
708342.608498
707697.789301
707064.075999
706441.151898
705828.713238
705226.468501
704634.137781
704051.452177
703478.1532

In [6]:
bc.baseline_rmse(reviews_train)

1.2242238409059312

In [7]:
bc.baseline_rmse(reviews_test)

1.2585870968573407

In [8]:
bc2 = L2RegLeastSquaresBaselineCalculator()
bc2.fit(reviews_train, l2_pen=0.1)

828243.574404
820860.237999
814343.119102
808515.925852
803247.342421
798438.234688
794012.719585
789911.887563
786089.356487
782508.096981
779138.145874
775954.944036
772938.116315
770070.566836
767337.801075
764727.412346
762228.68858
759832.307888
757530.100308
755314.859346
753180.191363
751120.394007
749130.357163
747205.481515
745341.611043
743534.976606
741782.148471
740079.996078
738425.653735
736816.491199
735250.088301
733724.212966
732236.802076
730785.944736
729369.867587
727986.921873
726635.571999
725314.38539
724022.023463
722757.23358
721518.84184
720305.746626
719116.91279
717951.366424
716808.190132
715686.518749
714585.535466
713504.468301
712442.586894
711399.199576
710373.650701
709365.318197
708373.611325
707397.968625
706437.856023
705492.765089
704562.211437
703645.733243
702742.889875
701853.260631
700976.443562
700112.054388
699259.725486
698419.104951
697589.855726
696771.654783
695964.192369
695167.171294
694380.306273
693603.323304
692835.959091
692077.9605

In [9]:
bc2.baseline_rmse(reviews_train)

1.2183229084612857

In [10]:
bc2.baseline_rmse(reviews_test)

1.2554672409037799